<a href="https://colab.research.google.com/github/amalvarezme/AnaliticaDatos/blob/master/2_Regresion/1_Guia_LadoaLado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Construyendo un proyecto de aprendizaje de máquina**

## Aplicación en regresión

*Códigos referencia del libro Hands-On Machine Learning with Scikit-Learn and Tensor Flow 2017 (Cap 2) -- Aurélien Géron

In [ ]:
#cargar datos desde drive acceso libre
FILEID = "1vrk5heTgxMBZagbw62SSVSWvfrW--Jgz"
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$FILEID -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id="$FILEID -O codigos.zip && rm -rf /tmp/cookies.txt
!unzip codigos.zip
!dir

# Etapas principales en un sistema de aprendizaje de máquina

1. Observar el problema (ayuda interdisciplinar)
2. Obtener los datos
3. *Análisis exploratorio incial (visualizar los datos y estadística descriptiva básica)*
4. *Preparar los datos para los algoritmos de aprendizaje de máquina (evaluación, preproceso, caracterización, aprendizaje)*
5. *Seleccionar un modelo y entrenar*
6. *Sintonizar el modelo escogido*
7. Presentar la solución
8. Lanzar la solución, monitorear y mantener el sistema de aprendizaje de máquina.

# Problema a resolver

**Objetivo**: predecir precios medios en distritos de [California Housing Prices](https://www.kaggle.com/camnugent/california-housing-prices).

**Insumos**: Características de los distritos.

# Inicio códigos

**Preparar modulos principales, funciones inline, paths para guardar archivos y figuras:**

In [ ]:
import numpy as np #importar librerias básicas
import os

# fijar semilla para fines pedagógicos
np.random.seed(42)

#render de figs sobre notebook
%matplotlib inline
# para graficar
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

#crear carpeta con resultados
import os
try:
  os.mkdir('results')
except:
  print("Carpeta results ya existe")

import warnings #eliminar warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Obtener los datos

In [ ]:
import pandas as pd #importar pandas
housing = pd.read_csv('datasets/housing/housing.csv')#leer archivo csv con pandas
housing.head() #encabezado principal

# Análisis exploratorio básico

In [ ]:
housing.info() #revisar cantidad de datos, tipos de datos, campos vacios

In [ ]:
housing["ocean_proximity"].value_counts() #conteo de los valores en una característica

In [ ]:
housing.describe() # estadística básica de las variables

In [ ]:
x  = np.array([-500,-2,0,0.5,4,10,1000]) #diferencias entre media y mediana
print(np.median(x),x.mean())

**Nota**: los [cuartiles](https://en.wikipedia.org/wiki/Interquartile_range) permiten revisar la asimetría de los datos.

# Indexación básica de usando pandas - DataFrame

In [ ]:
housing.index[0] #el indice se refiere a las filas (muestras)

In [ ]:
housing.loc[[0,4,5],['longitude','latitude']] #para indexar filas y columnas se pueden utilizar listas con los nombres de indices y columnas

In [ ]:
housing.loc[housing.index[[0,2,3]]] #los indices están en la variable index y se utiliza el método loc

In [ ]:
housing[housing.columns[0:5]]#las columnas (atributos estan en la variable columns) por defecto se refieren como los keys de un diccionario

In [ ]:
housing.iloc[0:5,4:7] # con el método iloc se puede indexar igual que en la librería numpy

In [ ]:
housing.loc[0] #se indexa el primer elemento de la matriz

In [ ]:
housing['latitude'] #se indexa la variable latitude

In [ ]:
housing.to_numpy() #to_numpy() convierte el dataframe a numpy array

In [ ]:
housing["median_house_value"].describe() # estadística básica de las variable de interés

In [ ]:
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20,15))
plt.tight_layout()
plt.savefig('results/attribute_histogram_plots.pdf', format='pdf', dpi=300)#para grabar resultados en pdf
plt.show()

# **Ejercicio 1**

- Discuta los histogramas visualizados.
- Respecto a la variable de interés a predecir, qué puede inferer del histograma?

# Preparar datos

- Incluye análisis exploratorio y preprocesamiento referente a campos vacios y datos tipo categóricos.

**Nota**:

Se recomienda realizar una partición training/testing para no sobreajustar los análisis a los datos de training

![CV](https://github.com/amalvarezme/AnaliticaDatos/blob/master/2_Regresion/crossvalidation.png?raw=1)




In [ ]:
from sklearn.model_selection import train_test_split #particionar

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)#20% para testing, dejando semilla fija

In [ ]:
print(train_set.shape,test_set.shape)#se revisa el tamaño de los sets

In [ ]:
test_set.head()

In [ ]:
housing["median_income"].hist()
plt.show()

**Preproceso variable median_income -> continua a categórica**

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5]) # se umbraliza por rangos la variable median income

In [ ]:
housing["income_cat"].value_counts()

In [ ]:
housing["income_cat"].hist()
plt.show()

**Nota**: para evitar problemas a la hora de particionar los datos, se recomiendo utilizar muestreo estratificado cuando hay distribuciones asimétricas en las distribuciones de las variables (desbalance).

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit #método para participación estratificado

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"].values): #income_cat es la variable a utilizar para estratificar
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
test_index #split define la fila a escoger durante la partición

In [ ]:
strat_train_set["income_cat"].value_counts() / len(strat_train_set)#conteo datos estratificados en train

In [ ]:
strat_test_set["income_cat"].value_counts() / len(strat_test_set)#conteo datos estratificados en test

In [ ]:
housing["income_cat"].value_counts() / len(housing) #conteo datos originales

**Comparar errores en particiones con y sin estratificación**

In [ ]:
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({ #se crea diccionario para revisar porcentajes
    "Overall": income_cat_proportions(housing),
    "Stratified": income_cat_proportions(strat_test_set),
    "Random": income_cat_proportions(test_set),
}).sort_index()
compare_props["Rand. %error"] = 100 * compare_props["Random"] / compare_props["Overall"] - 100
compare_props["Strat. %error"] = 100 * compare_props["Stratified"] / compare_props["Overall"] - 100

In [ ]:
compare_props

In [ ]:
for set_ in (strat_train_set, strat_test_set): #retirar filas y columnas
    set_.drop("income_cat", axis=1, inplace=True)
#strat_train_set.drop("income_cat", axis=1, inplace=True) #se puede realizar elemento a elemento
#strat_test_set.drop("income_cat", axis=1, inplace=True)

# Análisis exploratorio y visualización sobre datos muestreados

In [ ]:
housing = strat_train_set.copy() #se copia los datos estratificados
housing.shape

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude") #visualización georeferenciada
plt.tight_layout()
plt.savefig('results/bad_visualization_plot.pdf', format='pdf', dpi=300)
plt.show()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1) #alpha controla la trasparencia
plt.tight_layout()
plt.savefig('results/bad_visualization_plot.pdf', format='pdf', dpi=300)
plt.show()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.4,
    s=housing["population"]/100, label="population", figsize=(10,7),
    c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)
plt.legend()
plt.tight_layout()
plt.savefig('results/housing_prices_scatterplot.pdf', format='pdf', dpi=300)
plt.show()

In [ ]:
#definir mapa y disperción
import matplotlib.image as mpimg
california_img=mpimg.imread('datasets/california.png')
ax = housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7),
                       s=housing['population']/100, label="Population",
                       c="median_house_value", cmap=plt.get_cmap("jet"),
                       colorbar=False, alpha=0.4,
                      )
plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5,
           cmap=plt.get_cmap("jet"))
plt.ylabel("Latitude", fontsize=14)
plt.xlabel("Longitude", fontsize=14)

prices = housing["median_house_value"]
tick_values = np.linspace(prices.min(), prices.max(), 11)
cbar = plt.colorbar()
cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
cbar.set_label('Median House Value', fontsize=16)

plt.legend(fontsize=16)
plt.tight_layout()
plt.savefig('results/california_housing_prices_plot.pdf', format='pdf', dpi=300)
plt.show()

# **Ejercicio 2**

- Discuta sobre la visualización de los datos georreferenciados respecto a las variables consideradas



**Identificación de correlaciones**

La [correlación](https://en.wikipedia.org/wiki/Correlation) es una de las medidas básicas para definir relaciones lineales entre atributos.

$R_{xy} = \mathbb{E}\{xy\}$: correlación

$\sigma_{xy} = \mathbb{E}\{(x-\mu_x)(y-\mu_y)\}$ : covarianza

**Coeficiente de Pearson**:

$\rho_{xy} = \frac{\sigma_{xy} }{\sigma_{x}\sigma_y} \in [-1,1]$

$\sigma^2_x = \mathbb{E}\{(x-\mu_x)^2\}$: varianza

$\sigma_x$ : desviación estandar (std)

In [ ]:
corr_matrix = housing.corr() #función para calcular las correlaciones entre las variables
corr_matrix.style.background_gradient(cmap='coolwarm')

In [ ]:
import seaborn as sns #librería útil para graficar mapas de calor, matrices
sns.heatmap(corr_matrix,xticklabels=corr_matrix.columns.values,yticklabels=corr_matrix.columns.values)
plt.tight_layout()
plt.savefig('results/corr_matrix.pdf', format='pdf', dpi=300)
plt.show()


In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False) #orgnizar correlaciones con variable a predecir

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age","latitude"]
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.tight_layout()
plt.savefig('results/scatter_matrix_plot.pdf', format='pdf', dpi=300)
plt.show()

**Generar nuevas características intuitivas**

En muchas ocasiones relaciones entre las variables pueden generar nuevas características (ingeniería de características).

In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["population_per_household"]=housing["population"]/housing["households"]

In [ ]:
housing.head()

In [ ]:
corr_matrix = housing.corr()#correlación incluyendo nuevas variables
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing.describe()

# Preparar los datos para los algoritmos de aprendizaje de máquina

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # eliminar variable de salida de las entradas
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
housing.isnull().any(axis=1) # cualquier columna con alguna fila en null

In [ ]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1) ]
sample_incomplete_rows

In [ ]:
sample_incomplete_rows.shape

In [ ]:
tmp = housing[(housing.isnull().any(axis=1)) & (housing['latitude'] > 35)]
tmp

# Opciones básicas para lidiar con datos perdidos

1. Quitar filas (instancias con característica perdida)
2. Quitar columnas (se elimina la característica completa si presenta datos perdidos)
3. Se estiman los valores perdidos mediante mediana, promedio, moda, o estimaciones por vecindario.

In [ ]:
sample_incomplete_rows.dropna(subset=["total_bedrooms"])    # opción 1: eliminar filas

In [ ]:
sample_incomplete_rows.drop("total_bedrooms", axis=1)       # opción 2: eliminar columnas

In [ ]:
median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True) # opción 3: estimar datos perdidos
sample_incomplete_rows

**Clase Imputer de scikit-learn contiene este preproceso para datos perdidos**

In [ ]:
from sklearn.impute import SimpleImputer  # clase para completar datos perdidos
imputer = SimpleImputer(strategy="median") #crear objeto con estimador de mediana

Se deben remover las variables nominales para aplicar la opción por mediana:

In [ ]:
housing_num = housing.drop('ocean_proximity', axis=1)


In [ ]:
imputer.fit(housing_num) #se ajusta el modelo de completar datos numéricos

In [ ]:
imputer.statistics_ #valores obtenidos en preproceso imputer por atributo numérico->mediana

Dichos valores son las mismas medianas:

In [ ]:
housing_num.median().values

Aplicar el preproceso (transformación) según lo entrenado con imputer:

In [ ]:
X = imputer.transform(housing_num)#se aplica el modelo de completar datos
X #el transformer devuleve un array

In [ ]:
X = imputer.fit_transform(housing_num)

In [ ]:
housing_num_imp = pd.DataFrame(X, columns=housing_num.columns)
                          #index = list(housing.index.values)) #se puede volver a pandas frame

In [ ]:
housing_num_imp.isnull().sum()

In [ ]:
imputer.strategy

# Cómo trabajar con variables nominales (categóricas)? (ej: 'ocean_proximity')

1. **Etiquetar (codificar) los textos**. Problemas: la noción de cercanía pierde sentido
2. **One-hot-encoding** una nueva característica binaria se genera por cada categoría de la variable de interés. Problema: el número de características crece considerablemente.

In [ ]:
housing_cat = housing[['ocean_proximity']]#ejemplo con ocean proximity
housing_cat.head(10)

In [ ]:
np.unique(housing_cat)

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)#se entrena y evalua en cascada
#ordinal_encoder.fit(housing_cat) #entreno
#housing_cat_encoded = ordinal_encoder.transform(housing_cat) #transformar/evaluar
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_#codificación según orden implementado

In [ ]:
from sklearn.preprocessing import OneHotEncoder

cat_encoder = OneHotEncoder(sparse=True)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

Por defecto, la clase `OneHotEncoder` retorna un arreglo ralo (sparse: la mayoría de elementos en 0), se puede usar la función `toarray()` para trabajar con arreglos completos (full).

In [ ]:
housing_cat_1hot.toarray()

También se puede fijar la opción  `sparse=False` en `OneHotEncoder`:

In [ ]:
cat_encoder = OneHotEncoder(sparse=False)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

# Esquema general (pipeline) de preproceso para atributos numéricos

1. Lidiar con datos perdidos -> SimpleImputer
2. Escalado de atributos (min max ; estandarización - zscore). Min-max sensible a atípicos  

In [ ]:
#datos numéricos
housing_num_imp.head()

In [ ]:
#pandas datos categóricos
housing_cat_ord = pd.DataFrame(housing_cat_encoded,columns=['Ocean_encoded'])
housing_cat_ord.head()

In [ ]:
#pandas datos categóricos 1 hot encoder
housing_cat_1hot = pd.DataFrame(housing_cat_1hot,columns=list(range(housing_cat_1hot.shape[1])))
housing_cat_1hot.head()

In [ ]:
housing_cat_1hot.shape

In [ ]:
print(housing_num_imp.shape,housing_cat_ord.shape)

In [ ]:
housing_cat_ord.isnull().sum()

In [ ]:
housing_tr = pd.concat([housing_num_imp,housing_cat_ord],axis=1)#concatenar pandas numéricos y categóricos sobre columnas
housing_tr.shape

In [ ]:
housing_tr.head()

In [ ]:
print(housing_tr.shape,housing_num.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(housing_tr)
housing_tr_nor = scaler.transform(housing_tr)

In [ ]:
housing_tr_num = pd.DataFrame(housing_tr_nor,columns=housing_tr.columns)
housing_tr_num.describe()

In [ ]:
housing_tr_num.head()

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

my_pipeline = Pipeline([
        ('std_scaler', StandardScaler()), #MinMaxScaler StandardScaler
        ('reg',LinearRegression())
    ])


# Seleccionar un modelo y entrenar sobre los datos preparados

In [ ]:
#entrenar el pipeline completo
my_pipeline.fit(housing_tr,housing_labels)

In [ ]:
output_e = my_pipeline.predict(housing_tr)#estimación de la salida sobre modelo entrenado
output_e.shape

Entrenar sobre un subconjunto de datos:

In [ ]:
# evaluar sobre algunos datos
some_data = housing_tr.loc[:5]
some_labels = housing_labels.loc[:5]

print("Predicciones:", my_pipeline.predict(some_data))

Comparar con datos verdaderos:

In [ ]:
print("Labels:", list(some_labels))

In [ ]:
some_data

In [ ]:
from sklearn.metrics import mean_squared_error #se cálcula el ajuste entre las predicciones y los datos usando el mse
lin_mse = mean_squared_error(housing_labels, output_e)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

Cuantificar rendimiento con error absoluto medio:

In [ ]:
from sklearn.metrics import mean_absolute_error

lin_mae = mean_absolute_error(housing_labels, output_e)
lin_mae

**Nota:** el mse y el mae se basan en el concepto norma-2 y norma-1 del error:

$${\rm{mse}}(\mathbf{y},\hat{\mathbf{y}})=\|\mathbf{y} -\hat{\mathbf{y}}\|_2$$

$${\rm{mae}}(\mathbf{y},\hat{\mathbf{y}})=\|\mathbf{y} -\hat{\mathbf{y}}\|_1$$


# **Ejercicio 3**

- Repetir el proceso expuesto para codificación OneHotEncoder y la normalización MinMaxScaler.

- Compare y discuta los resultados en términos del mse y el mae.

In [ ]:
#Ayuda de código:
#concatenar datos numericos completos con codificacion onehot de ocean proximity
housing_tr2 = pd.concat([housing_num_imp,housing_cat_1hot],axis=1)#concatenar pandas numéricos y categóricos sobre columnas
housing_tr2.shape

# Sintonizar el modelo

Debemos verificar que el modelo no se este sobreajustando: i) necesitamos más datos? ii) un modelo más simple? iii) restringirlo (regularizarlo)?

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

reg = RandomForestRegressor()
#se estiman el rendimiendo sobre el conjunto de test para 10 particiones
#se presenta el score del conjunto de test en cada partición
scores = cross_val_score(reg, housing_tr, housing_labels,
                         scoring="neg_mean_squared_error", cv=5,verbose=100,n_jobs=-1) #scikitlearn trabaja con función util (mayor mejor) no función de costo (menor mejor)
tree_rmse_scores = np.sqrt(-scores)
#verbose presenta en pantalla la evolución del proceso
#n_jobs = -1 permite paralelizar sobre cpu

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
reg_lin = LinearRegression()#se prueba sobre un regresor lineal
scores = cross_val_score(reg, housing_tr, housing_labels,
                         scoring="neg_mean_squared_error", cv=5,verbose=100,n_jobs=-1)

lin_scores = cross_val_score(reg_lin, housing_tr, housing_labels,
                             scoring="neg_mean_squared_error", cv=10,verbose=100,n_jobs=-1)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Búsqueda de hyperparámetros por regilla en validación cruzada:

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # se evaluan 12 (3×4) combinaciones de hiperparámetros
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # se evaluan 6 (2×3) combinaciones
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# se generan 5 folds, para un total de (12+6)*5=90 entrenamientos
grid_search = GridSearchCV(forest_reg, param_grid, cv=5, verbose=100,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_tr, housing_labels)

Los mejores hyperparámetros encontrados:

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

Observemos los hyperparámetros escogidos durante las particiones:

In [ ]:
cvres = grid_search.cv_results_#revisar los resultados por cv
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV #CV aleatorizada
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),#regilla con muestreo aleatorio bajo distribución uniforme
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,verbose=100,n_jobs=-1,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_tr, housing_labels)

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

# Material extra

## Pipeline completo para preparación de datos y predicción

In [ ]:
full_pipeline_with_predictor = Pipeline([
        ("std_sca", StandardScaler()),
        ("linear", LinearRegression())
    ])

full_pipeline_with_predictor.fit(housing_tr, housing_labels)
full_pipeline_with_predictor.predict(some_data)

## Creación de modelo eficiente con joblib

In [ ]:
my_model = full_pipeline_with_predictor

In [ ]:
import joblib #https://joblib.readthedocs.io/en/latest/
joblib.dump(my_model, "results/my_model.pkl") # DIFF  https://docs.python.org/2/library/pickle.html
#...
my_model_loaded = joblib.load("results/my_model.pkl") # DIFF

## Ejemplos de distribuciones en SciPy para `RandomizedSearchCV`

In [ ]:
from scipy.stats import geom, expon
#algunas distribuciones desde scipy para buscar hiperparámetros
geom_distrib=geom(0.5).rvs(10000, random_state=42)
expon_distrib=expon(scale=1).rvs(10000, random_state=42)
plt.hist(geom_distrib, bins=50)
plt.show()
plt.hist(expon_distrib, bins=50)
plt.show()

In [ ]:
from sklearn.linear_model import ElasticNet

Elastic_reg = ElasticNet(random_state=0)
Elastic_reg.fit(housing_tr, housing_labels)
#GridCV RandomCV
housing_predictions = Elastic_reg.predict(housing_tr)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
#guardar resultados
from google.colab import files
from datetime import date, datetime
import shutil
#guardar resultados
namefile = str(datetime.now().strftime("%Y_%m_%d_%H_%M_%d"))+'__results'
shutil.make_archive(namefile, 'zip', 'results')
files.download(namefile+'.zip')